In [1]:
# from neural_odes import *
# %load_ext autoreload
# %autoreload 2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from neural_odes import *
from tqdm import tqdm_notebook as tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Solve the Discrete Lotka-Volterra Eqtn for general non-symmetric $A$
\begin{equation}
    \begin{aligned}
    p_i(t+1) &= p_i(t)\big[1+r_i\big(1-\frac{\sum_{j=1}^dA_{ij}p_j(t)}{k_i}\big)\big], i = 1, \dots d\\
    &= p_i(t)\big[1+r_i\big(1-\frac{\mathbf{b}_i^T\big(\sum_{j=1}^d\mathbf{c}_jp_j(t)\big)}{k_i}\big)\big], i = 1, \dots d \\
    &= p_i(t)\big[1+r_i\big(1-\frac{\mathbf{b}_i^TC\mathbf{p}}{k_i}\big)\big], i = 1, \dots d \\
    \end{aligned}
\end{equation}.

We approximate $A_{ij} = \mathbf{b}_i^T\mathbf{c}_j$ using the low rank matrix approximation $A= B^TC$, where $B = [\mathbf{b}_1, \cdots, \mathbf{b}_d] \in \mathbb{R}^{k \times d}$ and $C = [\mathbf{c}_1, \cdots, \mathbf{c}_d] \in \mathbb{R}^{k \times d}$.  Each $\mathbf{b}_i, \mathbf{c}_i \in \mathbb{R}^k$, where $k \ll d$ are the embeddings of time series $i$.

In matrix-vector form, we have $A\mathbf{p}$, which has computational complexity $\mathcal{O}(d^2)$ for $A \in \mathbb{R}^{d \times d}, \mathbf{p} \in \mathbb{R}^d$.  Using the low-rank form, we can write $A\mathbf{p} = B^T(C \mathbf{p})$.  We do not want to explicitly form the matrix $B^TC$, since this would have higher complexity of $\mathcal{O}(kd^2)$.  We instead break the computation into two matrix-vector products as indicated by the parathesis, each of complexity $\mathcal{O}(kd) \ll \mathcal{O}(d^2).$ 

We start with $d = 100$ for the number of time series and will learn the synthetic data from the equation for random initialized $A, \mathbf{r}, \mathbf{k}$

### Solve the LV eqn for $p_i(t+1), 0 \le t < N - 1$.  We store $P$ as a matrix in $\mathbb{R}^{d \times N}$, whose first column is the initial condition $\mathbf{p}(0) \in \mathbb{R}^d$. Then $P = [\mathbf{p}(0), \dots, \mathbf{p}(N-1)].$

### Simple Equation Test Case: symmetric

In [2]:
num_time_steps = 2
num_time_series = 3

In [3]:
p0 = torch.cuda.FloatTensor([1,3,0])
k = torch.cuda.FloatTensor([2,3,1])
B = torch.cuda.FloatTensor([[1, 0, 1], [2, 1, 1]])
r = torch.cuda.FloatTensor([1,2,4])
A = torch.mm(B.transpose(1,0), B)

In [5]:
is_sym = True
is_full_matrix = True
low_rank_param = 1

In [6]:
B, A

(tensor([[1., 0., 1.],
         [2., 1., 1.]], device='cuda:0'), tensor([[5., 2., 3.],
         [2., 1., 1.],
         [3., 1., 2.]], device='cuda:0'))

In [33]:
# test formula matches for i = 0
((1 + r[0] * (1 - (A[0,0]*p0[0] + A[0,1] * p0[1] + A[0,2] * p0[2]) / k[0])) * p0[0]).cpu().data.numpy()

array(-3.5, dtype=float32)

In [34]:
# test formula matches for i = 1
((1 + r[1] * (1 - (A[1,0]*p0[0] + A[1,1] * p0[1] + A[1,2] * p0[2]) / k[1])) * p0[1]).cpu().data.numpy()

array(-0.99999976, dtype=float32)

In [35]:
# test formula matches for i = 2
((1 + r[2] * (1 - (A[2,0]*p0[0] + A[2,1] * p0[1] + A[2,2] * p0[2]) / k[2])) * p0[2]).cpu().data.numpy()

array(-0., dtype=float32)

In [7]:
neural_lv = NeuralLV(num_time_series, num_time_steps, low_rank_param, is_full_matrix, p0, r, k, A, is_sym)

In [8]:
p = neural_lv.solve_discrete_lv(A)
p.cpu().data.numpy()

AttributeError: 'NeuralLV' object has no attribute 'k'

In [41]:
p_low_rank_sym = neural_lv.solve_discrete_lv(B, is_full_matrix=False)
p_low_rank_sym.cpu().data.numpy()

array([[ 1.        , -3.5       ],
       [ 3.        , -0.99999976],
       [ 0.        , -0.        ]], dtype=float32)

In [42]:
C = None # symmetric case
# Check case where explicitly forming A too
A = compute_low_rank_product(B, C)
p_low_rank_prod_sym = neural_lv.solve_discrete_lv(A, is_full_matrix=True)
p_low_rank_prod_sym.cpu().data.numpy()

array([[ 1.        , -3.5       ],
       [ 3.        , -0.99999976],
       [ 0.        , -0.        ]], dtype=float32)

### Simple Equation Test Case: nonsymmetric

In [43]:
neural_lv = NeuralLV(num_time_series, num_time_steps, low_rank_param, is_full_matrix, p0, r, k, A, is_sym)

In [45]:
C = torch.cuda.FloatTensor([[0, 2, 3], [-1, -2, 0]])
A = torch.mm(B.transpose(1,0), C)

In [46]:
p = neural_lv.solve_discrete_lv(A)
p.cpu().data.numpy()

array([[ 1.,  6.],
       [ 3., 23.],
       [ 0.,  0.]], dtype=float32)

In [47]:
p_low_rank_nonsym = neural_lv.solve_discrete_lv(B, C, is_full_matrix=False)
p_low_rank_nonsym.cpu().data.numpy()

array([[ 1.,  6.],
       [ 3., 23.],
       [ 0.,  0.]], dtype=float32)

In [48]:
# Check case where explicitly forming A too
A = compute_low_rank_product(B, C)
p_low_rank_prod_sym = neural_lv.solve_discrete_lv(A, is_full_matrix=True)
p_low_rank_prod_sym.cpu().data.numpy()

array([[ 1.,  6.],
       [ 3., 23.],
       [ 0.,  0.]], dtype=float32)

### 4D LV Example of chaotic competitive LV systems
In this section, we verify that the equation solver works on this common LV test case (See https://en.wikipedia.org/wiki/Competitive_Lotka–Volterra_equations).

In [9]:
num_time_series = 4
num_time_steps = 50
low_rank_param = 5
is_sym = False
is_full_matrix = False
print(f'The number of time series = {num_time_series}')
print(f'The number of time steps = {num_time_steps}')

The number of time series = 4
The number of time steps = 50


In [10]:
torch.manual_seed(4)
_, _, p0, A = generate_data(num_time_series)
# r = torch.randn(num_time_series).float().to(device)
k = torch.ones(num_time_series).float().to(device)

r = torch.cuda.FloatTensor([1.0, 0.72, 1.53, 1.27])
A = torch.cuda.FloatTensor([[1.0, 1.09, 1.52, 0.0], [0.0, 1.0, 0.44, 1.36], 
                             [2.33, 0.0, 1.0, 0.47], [1.21, 0.51, 0.35, 1.0]])
# k = torch.abs(torch.randn(num_time_series).float().cuda())
p0 = torch.ones(num_time_series).float().cuda()/10
r.shape, k.shape, p0.shape, A.shape

(torch.Size([4]), torch.Size([4]), torch.Size([4]), torch.Size([4, 4]))

## Use embedding to learn $\mathbf{b}_i \in \mathbb{R}^k$

We are testing low rank symmetric case even though $A$ is not symmetric.  We will see in another notebook how computing two low rank matrices for non-symmetric $A$ improves the learning and convergence.

In [1]:
class LowRankVectorEmbedding(nn.Module):
    def __init__(self, neural_lv):
        super(LowRankVectorEmbedding, self).__init__()
        self.num_time_series = neural_lv.num_time_series # num_ts
        self.low_rank_param = neural_lv.low_rank_param # low rank parameter k
        self.is_full_matrix = neural_lv.is_full_matrix
        self.feat_static_cat = torch.arange(self.num_time_series).to(device)
        self.is_sym = neural_lv.is_sym
        self.neural_lv = neural_lv        
      
        self.embedding_low_rank_mat_B = nn.Embedding(self.num_time_series, self.low_rank_param)
        self.embedding_low_rank_mat_B.weight.data.uniform_(-0.1, 0.1)
        if not self.is_sym:
            self.embedding_low_rank_mat_C = nn.Embedding(self.num_time_series, self.low_rank_param)
            self.embedding_low_rank_mat_C.weight.data.uniform_(-0.1, 0.1)
    def forward(self):
        # find low rank vector computed per time series
        # feat_static_cat consists of the time series indices 0, ..., cardinality - 1
        # embedding returns (num_ts, low_rank_param) need to transpose it
        B = self.embedding_low_rank_mat_B(self.feat_static_cat).T
        C = None if self.is_sym else self.embedding_low_rank_mat_C(self.feat_static_cat).T
        # Explicitly form matrix matrix product A = B^T* CO(kd^2) expensive
        if self.is_full_matrix:
            return self.neural_lv.solve_discrete_lv(compute_low_rank_product(B, C))
        # Compute matrix vector product B^T * (C p) O(kd)
        else:
            return self.neural_lv.solve_discrete_lv(B, C, self.is_full_matrix)


NameError: name 'nn' is not defined

In [2]:
class NeuralLV(nn.Module):
    def __init__(self, num_time_series, num_time_steps, low_rank_param, 
                 is_full_matrix, p0, r, k, A, is_sym):
        super(NeuralLV, self).__init__()
        # Define number of time series
        self.num_time_series = num_time_series
        # Define number of discrete time steps will assume the same for each time series so p_i(t) in R^(dxN)
        self.num_time_steps = num_time_steps
        self.low_rank_param = low_rank_param
        self.is_full_matrix = is_full_matrix
        self.p0 = p0
        self.true_r = r
        self.r = nn.Parameter(torch.rand(num_time_series).float().cuda())#r
        self.true_k = k
        self.k = nn.Parameter(torch.rand(num_time_series).float().cuda())
        self.A = A
        self.is_sym = is_sym
        
        
        self.feat_static_cat = torch.arange(self.num_time_series).to(device)
        
        #self.neural_lv = neural_lv        
      
        self.embedding_low_rank_mat_B = nn.Embedding(self.num_time_series, self.low_rank_param)
        self.embedding_low_rank_mat_B.weight.data.uniform_(-0.1, 0.1)
        if not self.is_sym:
            self.embedding_low_rank_mat_C = nn.Embedding(self.num_time_series, self.low_rank_param)
            self.embedding_low_rank_mat_C.weight.data.uniform_(-0.1, 0.1)
    
    
    def solve_discrete_lv(self, mat1, mat2=None, is_full_matrix=True, is_target = False):
        p = [] # need to store as list for autograd won't let you append indices in same matrix
        p.append(self.p0)
        for n in range(self.num_time_steps-1): # element-wise vector division and multiplication
            # Compute Ap to generate synthetic data for the full rank matrix A
            if is_full_matrix:
                #print(mat1.shape, p[n].shape)
                mat_vec_prod = torch.mm(mat1, p[n].reshape(-1, 1)).squeeze(-1)
            else:
                mat_vec_prod = compute_mat_vec_prod(mat1, mat2, p[n].reshape(-1, 1)).squeeze(-1)
            if is_target:
                p.append((1 + self.true_r * (1 - mat_vec_prod / self.true_k)) * p[n])
            else:
                p.append((1 + self.r * (1 - mat_vec_prod)) * p[n])#/ self.k

        return torch.cat(p, dim=0).reshape(self.num_time_steps, self.num_time_series).T
    
    def forward(self):
        B = self.embedding_low_rank_mat_B(self.feat_static_cat).T
        C = None if self.is_sym else self.embedding_low_rank_mat_C(self.feat_static_cat).T
        
        if self.is_full_matrix:
            p_approx = self.solve_discrete_lv(compute_low_rank_product(B, C))
        else:
            p_approx = self.solve_discrete_lv(B, C, self.is_full_matrix)
        p = self.solve_discrete_lv(self.A, is_target = True)
        return p, p_approx, compute_low_rank_product(B, C)


NameError: name 'nn' is not defined

In [3]:
num_time_series = 4
num_time_steps = 10
is_sym = False
is_full_matrix = True
low_rank_param = 1

In [14]:
model = NeuralLV(num_time_series, num_time_steps, low_rank_param, is_full_matrix, p0, r, k, A, is_sym).to(device)
optimizer = torch.optim.Adam(model.parameters(), 0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1, gamma= 0.9)
loss_fun = torch.nn.MSELoss()#SmoothL1Loss()#
tqdm_epochs = tqdm(range(30))
for e in tqdm_epochs:
    p, p_approx, A_approx = model.forward()
    Loss = loss_fun(p, p_approx)
    optimizer.zero_grad()
    Loss.backward()#retain_graph=True
    optimizer.step()
    tqdm_epochs.set_postfix({'loss': Loss.item()})

AttributeError: 'NeuralLV' object has no attribute 'parameters'

In [11]:
max_num_plots=10
num_rows=2
fig_size_width=10
plt.rcParams["figure.figsize"] = (fig_size_width, 5)
num_ts = p.shape[0]
N = p.shape[1]
t = np.arange(N)
num_plots = min(num_ts, max_num_plots)
num_cols = int(num_plots / num_rows)
fig, axs = plt.subplots(num_rows, num_cols)
for ts_idx in range(num_plots):
    plt.subplot(num_rows, num_cols, ts_idx+1) 
    plt.plot(t, p[ts_idx, :].cpu().data.numpy(), \
             t, p_approx[ts_idx, :].cpu().data.numpy(), 'r--')
    plt.ylabel(f'$p_{ts_idx}(t)$')
    plt.xlabel('t')
    plt.legend(('Exact', 'Approx'))
    plt.xlabel('time: $t$')
    plt.ylabel(f'$p_{ts_idx}(t)$')
#plt.savefig("learn_k_r_10_nonsym.png", dpi = 300 , bbox_inches = "tight")

NameError: name 'p' is not defined

## Compute Errors and Plots

In [69]:
print(f'l2 norm of the error = {torch.sqrt(torch.mean((p_approx-p)**2))}')

l2 norm of the error = 0.011699080467224121


In [70]:
print(f'max norm of the error = {torch.max(torch.abs(p_approx-p))}')

max norm of the error = 0.03163820505142212


In [10]:
print(f'l2 matrix norm of the error of A and its low rank approx = {torch.sqrt(torch.mean((A_approx-A)**2))}')

l2 matrix norm of the error of A and its low rank approx = 0.6239166259765625


In [29]:
model.r

Parameter containing:
tensor([0.8065, 0.9372, 1.6270, 1.6514], device='cuda:0', requires_grad=True)

In [11]:
model.A

tensor([[1.0000, 1.0900, 1.5200, 0.0000],
        [0.0000, 1.0000, 0.4400, 1.3600],
        [2.3300, 0.0000, 1.0000, 0.4700],
        [1.2100, 0.5100, 0.3500, 1.0000]], device='cuda:0')

In [16]:
A_approx

tensor([[ 2.1903,  1.4413,  1.1354, -0.8410],
        [ 2.3190,  1.5260,  1.2021, -0.8905],
        [ 2.5763,  1.6953,  1.3355, -0.9893],
        [ 2.3638,  1.5555,  1.2253, -0.9077]], device='cuda:0',
       grad_fn=<MmBackward>)

In [15]:
lv_plot_ts(p, p_approx)

NameError: name 'p' is not defined